In [ ]:
pip install deepface

In [ ]:

pip install mysql.connector

In [ ]:
#Getting the data from the GCP Users Sample
import cv2
from PIL import Image
import urllib.request
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#importing deepface library and DeepFace
from deepface import DeepFace
import mysql.connector
from datetime import datetime
import time


print('start at: ' + str(datetime.now()))


mydb = mysql.connector.connect(
            host = "localhost",
            user = "root",
            password = "mysql@123",
            database = "AirBnb"
        )
cursor = mydb.cursor()
#query = "SELECT * FROM airbnb_listing_host where picture_large_url is not null limit 0, 1000;"
query = "SELECT * FROM AirBnb.airbnb_listing_users_sample;"
cursor.execute(query)
 
myresult = cursor.fetchall()

print(myresult)


In [ ]:

model_name = "VGG-Face"
model = DeepFace.build_model(model_name)


In [ ]:
#Face analysis using Deep Face
count = 0
df = pd.DataFrame(myresult)
print(df.columns.values)
for x in myresult:
    print(x)
    url = x[3]
    try:
      req = urllib.request.urlopen(url)
      arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
      img = cv2.imdecode(arr, -1) # 'Load it as it is'
      try:
        prediction = DeepFace.analyze(img, enforce_detection=True)
        #time.sleep(10)
        if(str(prediction) != ""):
          g = str(prediction['gender'])
          a = str(prediction['age'])
          dr= str(prediction['dominant_race'])
          print('predicted dominant_race is: ' + str(dr))
          df.loc[count,[10]] = g
          df.loc[count,[11]] = a
          df.loc[count,[12]] = dr
      except :
        prediction = ""
        pass
    except:
      prediction = ""
      img = ""
      pass
    count+=1



In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
#filteredHosts = hosts[hosts[13].notnull()]
#hosts.where(hosts[13])
#print(filteredHosts)
#print(filteredHosts)
print(myresult)
print(df)

In [ ]:
deepFaceResultset = pd.DataFrame(df);
print(deepFaceResultset.columns.values)

In [ ]:
deepFaceResultset.drop(deepFaceResultset.columns[[0, 1, 2, 3, 7, 8,9]], axis=1, inplace=True)
print(deepFaceResultset)

In [ ]:
#Deepface - Confusion matrix for gender prediction
confusion_matrix_gender = pd.crosstab(deepFaceResultset[4].str.lower(), deepFaceResultset[10].str.lower(), rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix_gender)

In [ ]:
#Deepface - Confusion matrix for ethnicity prediction
confusion_matrix_race = pd.crosstab(deepFaceResultset[6].str.lower(), deepFaceResultset[12].str.lower(), rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix_race)

In [ ]:
  
import requests

# function to add to JSON
def write_json(new_data, filename='betafaceResult.json'):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)
  
    

class faceData:
    def __init__(self):
        pass

    def getData(self, url, filename):
        headers = {
            'accept': 'application/json',
            'Content-Type': 'application/json',
        }

        data = '{ "api_key": "d45fd466-51e2-4701-8da8-04351c872236", "file_uri": "' + url + '", "detection_flags": "basicpoints,propoints,classifiers,content", "recognize_targets": [ "all@mynamespace" ], "original_filename": "' + filename + '"}'

        response = requests.post('https://www.betafaceapi.com/api/v2/media', headers=headers, data=data).json()

        return response

import json

count = 0
betaFaceResultset = pd.DataFrame(myresult)

for x in myresult:
    #print(x)
    url = x[2]
    print(url)

    try:
        f = faceData()
        data = f.getData(filename="sample.jpg", url=url)
        data2 = json.dumps(data)
        parsed = json.loads(data2)
        write_json(parsed)

        try:
            data = f.getData(filename="sample.jpg", url=url)
            data2 = json.dumps(data)
            parsed = json.loads(data2)
            tagCount = 0
            
            print(parsed['media']['faces'][0]['tags'])
            for i in parsed['media']['faces'][0]['tags']:
                if(parsed['media']['faces'][0]['tags'][tagCount]['name'] == 'age'):
                    betaFaceResultset.loc[tagCount,[11]] = str(parsed['media']['faces'][0]['tags'][tagCount]['value'])
                    print(str(parsed['media']['faces'][0]['tags'][tagCount]['value']))
                    
                elif(parsed['media']['faces'][0]['tags'][tagCount]['name'] == 'gender'):
                    betaFaceResultset.loc[tagCount,[10]] = str(parsed['media']['faces'][0]['tags'][tagCount]['value'])
                    print(str(parsed['media']['faces'][0]['tags'][tagCount]['value']))
                    
                elif(parsed['media']['faces'][0]['tags'][tagCount]['name'] == 'race'):
                    betaFaceResultset.loc[tagCount,[12]] = str(parsed['media']['faces'][0]['tags'][tagCount]['value'])
                    print(str(parsed['media']['faces'][0]['tags'][tagCount]['value']))
                    
                tagCount+=1
        except :
            print(data);
            print(parsed);
            data = ""
            img = ""
            parsed = ""
            pass
    except:
        print(data);
        print(parsed);
        data = ""
        img = ""
        pass
        count+=1



In [ ]:
betaFaceResultset.drop(betaFaceResultset.columns[[0, 1, 2, 3, 4, 5,6]], axis=1, inplace=True)
print(betaFaceResultset)

In [ ]:
confusion_matrix_gender = pd.crosstab(betaFaceResultset[7].str.lower(), betaFaceResultset[10].str.lower(), rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix_gender)

In [ ]:
confusion_matrix_race = pd.crosstab(betaFaceResultset[9].str.lower(), betaFaceResultset[12].str.lower(), rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix_race)